In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import cv2
from tensorflow.keras import regularizers
from tensorflow.keras import layers
import os

In [2]:
GESTURE_TYPES = 11
LABEL_DICT = {k:i for i,k in enumerate([21, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33])}
CONNECTION_LABELS = [
    (0, 1), (1, 2), (2, 3), (3, 4),
    (5, 6), (6, 7), (7, 8),
    (9, 10), (10, 11), (11, 12),
    (13, 14), (14, 15), (15, 16),
    (17, 18), (18, 19), (19, 20),
    (0, 5), (5, 9), (9, 13), (13, 17), (0, 17)
]

In [3]:
def load_data(data_path='gesture_recognition/Fall 2020/data_rotated', sequence_length=5):
    keypoints = []
    labels = []
    for subjectName in os.listdir(data_path):
        if not (subjectName.startswith("Subject") or subjectName.startswith("subject")): continue
        # subjectNum = int(re.findall(r'(\d+)', subjectName)[0])
        for sceneName in os.listdir(os.path.join(data_path, subjectName)):
            if not (sceneName.startswith("Scene") or subjectName.startswith("scene")): continue
            for groupEntry in os.scandir(os.path.join(data_path, subjectName, sceneName)):
                with open(groupEntry, 'r') as f:
                    groupData = json.load(f)
                    for gesture in groupData:
                        # print(gesture['label'], gesture['keypoints'])
                        for i in range(len(gesture['keypoints'])):
                            if not gesture['keypoints'][i]:
                                gesture['keypoints'][i] = [[np.nan, np.nan, np.nan] for _ in range(21)]
                        for start_frame in range(len(gesture['keypoints']) - sequence_length + 1):
                            keypoints.append(gesture['keypoints'][start_frame: start_frame + sequence_length])
                            labels.append(LABEL_DICT[gesture['label']])
    keypoints = np.array(keypoints)
    labels = np.array(labels)
    return keypoints, labels
keypoints, labels = load_data()

In [4]:
def generate_connection_angles(keypoints, sequence_length=5, keypoints_num=21, keypoints_dimensions=3):
    connections = []
    for connection in CONNECTION_LABELS:
        connections.append(keypoints[..., connection[1], :] - keypoints[..., connection[0], :])
    connections = np.stack(connections, axis = -2)
    tensor1 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,2,4,3)
    tensor2 = connections[..., np.newaxis].repeat(keypoints_num, -1).transpose(0,1,4,2,3)
    angles = (tensor1*tensor2).sum(axis=-1)/np.linalg.norm(tensor1,axis=-1)/np.linalg.norm(tensor2,axis=-1)
    angles = angles.transpose(2,3,0,1)[np.triu_indices(21, k = 1)].transpose(1,2,0)
    return np.arccos(angles)
angles = generate_connection_angles(keypoints)

In [6]:
def visualize_keypoints(keypoints):
    for sequence in keypoints:
        for points in sequence:
            img = np.zeros((256, 256, 3))
            for point in points:
                x, y, z = point
                if np.isnan(x):
                    continue
                cv2.circle(img, (int(x), int(y)), 4, (255, 0, 0), 2)
            for connection in CONNECTION_LABELS:
                if np.isnan(points[connection[1]][0]):
                    continue
                x0, y0, z0 = points[connection[0]]
                x1, y1, z1 = points[connection[1]]
                cv2.line(img, (int(x0), int(y0)), (int(x1), int(y1)), (0, 255, 0), 2)
            cv2.imshow("Key Points", img)
            key = cv2.waitKey(100)
            if key == 27:
                cv2.destroyAllWindows()
                cv2.waitKey(1) # cannot close window on macOS without this line
                return
visualize_keypoints(keypoints)

In [6]:
def process_features(keypoints, angles):
    data_length = keypoints.shape[0]
    sequence_length = keypoints.shape[1]
    keypoints = keypoints.reshape(data_length*sequence_length, -1)
    angles = angles.reshape(data_length*sequence_length, -1)
    features = np.concatenate((keypoints, angles), -1)
    df = pd.DataFrame(features)
    df = (df-df.mean())/df.std()
    df = df.fillna(0)
    features = df.to_numpy().reshape(data_length, sequence_length, -1)
    return features
X = process_features(keypoints, angles)
#normalizer = tf.keras.layers.experimental.preprocessing.Normalization()
#normalizer.adapt(X)
# X_train, X_val, y_train, y_val = train_test_split(processed_keypoints, labels, test_size=0.2, random_state=0)

In [7]:
model_lstm = tf.keras.Sequential([tf.keras.layers.LSTM(GESTURE_TYPES, activation=None), tf.keras.layers.Activation('sigmoid')])
model_lstm.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics='accuracy')
history = model_lstm.fit(X, labels, epochs=10, validation_split=0.2)

Epoch 1/10
584/584 [==============================] - 1s 2ms/step - loss: 1.2533 - accuracy: 0.7077 - val_loss: 1.0325 - val_accuracy: 0.7610
Epoch 2/10
584/584 [==============================] - 1s 2ms/step - loss: 0.7463 - accuracy: 0.8272 - val_loss: 0.8730 - val_accuracy: 0.7797
Epoch 3/10
584/584 [==============================] - 1s 2ms/step - loss: 0.6240 - accuracy: 0.8501 - val_loss: 0.8646 - val_accuracy: 0.7887
Epoch 4/10
584/584 [==============================] - 1s 2ms/step - loss: 0.5449 - accuracy: 0.8599 - val_loss: 0.7887 - val_accuracy: 0.8017
Epoch 5/10
584/584 [==============================] - 1s 2ms/step - loss: 0.4234 - accuracy: 0.8697 - val_loss: 0.7034 - val_accuracy: 0.8135
Epoch 6/10
584/584 [==============================] - 1s 2ms/step - loss: 0.3795 - accuracy: 0.8794 - val_loss: 0.7696 - val_accuracy: 0.8064
Epoch 7/10
584/584 [==============================] - 1s 2ms/step - loss: 0.3531 - accuracy: 0.8854 - val_loss: 0.7842 - val_accuracy: 0.8092
Epoch 

In [8]:
model_rnn = tf.keras.Sequential([tf.keras.layers.SimpleRNN(GESTURE_TYPES, activation=None), tf.keras.layers.Activation('sigmoid')])
model_rnn.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics='accuracy')
history = model_rnn.fit(X, labels, epochs=10, validation_split=0.2)

Epoch 1/10
584/584 [==============================] - 1s 2ms/step - loss: 1.5480 - accuracy: 0.4291 - val_loss: 1.4615 - val_accuracy: 0.4497
Epoch 2/10
584/584 [==============================] - 1s 1ms/step - loss: 1.2213 - accuracy: 0.5324 - val_loss: 1.1742 - val_accuracy: 0.5542
Epoch 3/10
584/584 [==============================] - 1s 1ms/step - loss: 0.7941 - accuracy: 0.7020 - val_loss: 0.7767 - val_accuracy: 0.7565
Epoch 4/10
584/584 [==============================] - 1s 1ms/step - loss: 0.5794 - accuracy: 0.8158 - val_loss: 0.7348 - val_accuracy: 0.7794
Epoch 5/10
584/584 [==============================] - 1s 1ms/step - loss: 0.5314 - accuracy: 0.8344 - val_loss: 0.7164 - val_accuracy: 0.7820
Epoch 6/10
584/584 [==============================] - 1s 1ms/step - loss: 0.5036 - accuracy: 0.8429 - val_loss: 0.7016 - val_accuracy: 0.7946
Epoch 7/10
584/584 [==============================] - 1s 1ms/step - loss: 0.4866 - accuracy: 0.8471 - val_loss: 0.7167 - val_accuracy: 0.7906
Epoch 

In [9]:
model_lstm2 = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(GESTURE_TYPES, activation=None)), tf.keras.layers.Activation('sigmoid')
])
model_lstm2.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics='accuracy')
history = model_lstm2.fit(X, labels, epochs=10, validation_split=0.2)

Epoch 1/10
584/584 [==============================] - 2s 3ms/step - loss: 1.2720 - accuracy: 0.7391 - val_loss: 0.9133 - val_accuracy: 0.7606
Epoch 2/10
584/584 [==============================] - 2s 3ms/step - loss: 0.5956 - accuracy: 0.8321 - val_loss: 0.7756 - val_accuracy: 0.7807
Epoch 3/10
584/584 [==============================] - 2s 3ms/step - loss: 0.4809 - accuracy: 0.8533 - val_loss: 0.8177 - val_accuracy: 0.7842
Epoch 4/10
584/584 [==============================] - 2s 3ms/step - loss: 0.4291 - accuracy: 0.8651 - val_loss: 0.7420 - val_accuracy: 0.7970
Epoch 5/10
584/584 [==============================] - 2s 3ms/step - loss: 0.3947 - accuracy: 0.8756 - val_loss: 0.7351 - val_accuracy: 0.8015
Epoch 6/10
584/584 [==============================] - 2s 3ms/step - loss: 0.3607 - accuracy: 0.8832 - val_loss: 0.6908 - val_accuracy: 0.8075
Epoch 7/10
584/584 [==============================] - 2s 3ms/step - loss: 0.3441 - accuracy: 0.8895 - val_loss: 0.7397 - val_accuracy: 0.8109
Epoch 

In [10]:
model_lstm4 = tf.keras.Sequential([tf.keras.layers.LSTM(128), layers.Dense(GESTURE_TYPES), layers.Activation('sigmoid')])
model_lstm4.compile(loss=tf.losses.SparseCategoricalCrossentropy(), optimizer=tf.optimizers.Adam(), metrics='accuracy')
history = model_lstm4.fit(X, labels, epochs=10, validation_split=0.2)

Epoch 1/10
584/584 [==============================] - 2s 4ms/step - loss: 0.5487 - accuracy: 0.8484 - val_loss: 0.5405 - val_accuracy: 0.8240
Epoch 2/10
584/584 [==============================] - 2s 4ms/step - loss: 0.2247 - accuracy: 0.9302 - val_loss: 0.5660 - val_accuracy: 0.8272
Epoch 3/10
584/584 [==============================] - 2s 4ms/step - loss: 0.1336 - accuracy: 0.9650 - val_loss: 0.5931 - val_accuracy: 0.8285
Epoch 4/10
584/584 [==============================] - 2s 4ms/step - loss: 0.0815 - accuracy: 0.9810 - val_loss: 0.6285 - val_accuracy: 0.8156
Epoch 5/10
584/584 [==============================] - 2s 4ms/step - loss: 0.0551 - accuracy: 0.9877 - val_loss: 0.6726 - val_accuracy: 0.8206
Epoch 6/10
584/584 [==============================] - 2s 4ms/step - loss: 0.0368 - accuracy: 0.9925 - val_loss: 0.6749 - val_accuracy: 0.8231
Epoch 7/10
584/584 [==============================] - 2s 4ms/step - loss: 0.0253 - accuracy: 0.9945 - val_loss: 0.7246 - val_accuracy: 0.8263
Epoch 